In [14]:
import numpy as np
import sqlite3
import os

main=0

base_dir = os.getcwd()
DB_PATH = os.path.join(base_dir, "data", "Main.db")

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

cursor.execute('SELECT * FROM BaseParameter WHERE baseValue=(?)', (main, ))
row = cursor.fetchone()

print(row)

single=row[1]
h=row[2]

(0, 3.5e-07, 7e-07)


In [15]:
N=int(250)*2
r=N*single
x=np.linspace(-(N/2),(N/2),N)

lambda0=0.8e-6
lambda1=0.6e-6

f=60e-6

ftx0 = -(2 * np.pi) / lambda0 * (np.sqrt(x**2 + f**2) - f)
ftx0 = np.mod(ftx0, 2 * np.pi)  # wrapTo2Pi

ftx1 = -(2 * np.pi) / lambda1 * (np.sqrt(x**2 + f**2) - f)
ftx1 = np.mod(ftx1, 2 * np.pi)  # wrapTo2Pi

In [ ]:
cursor.execute('SELECT angleIn1, angleIn5 FROM Parameter WHERE baseValue=(?)', (main, ))
rows = cursor.fetchall()
print(rows)

phase = [0,0]
diff = np.empty(len(rows))
INDEX = np.empty(len(ftx0))

for i in range(N):
    phase[0] = ftx0[i]
    phase[1] = ftx1[i]
    for j,row in enumerate(rows):
        diff[j] = np.abs(phase[0]-row[0])+np.abs(phase[1]-row[1])
    
    min_index = np.argmin(diff)
    min_value = diff[min_index]
    INDEX[i] = min_index

[(2.2326421521943827, 5.2624718802425985), (2.3055513798457477, 5.355615082812643), (2.33685621656347, 5.395359756192938), (2.33685621656347, 5.395359756192938), (2.33685621656347, 5.395359756192938), (2.3768338063207666, 5.446965515108369), (2.4289329063547944, 5.51293963783582), (2.4289329063547944, 5.51293963783582), (2.4289329063547944, 5.51293963783582), (2.4515134729821746, 5.54079848645615), (2.4515134729821746, 5.54079848645615), (2.4515134729821746, 5.54079848645615), (2.4515134729821746, 5.54079848645615), (2.4515134729821746, 5.54079848645615), (2.4691625499726775, 5.56351418653956), (2.5314805478666136, 5.641494852180908), (2.5314805478666136, 5.641494852180908), (2.5314805478666136, 5.641494852180908), (2.5760474010453254, 5.696140186967039), (2.5760474010453254, 5.696140186967039), (2.5760474010453254, 5.696140186967039), (2.5760474010453254, 5.696140186967039), (2.5760474010453254, 5.696140186967039), (2.597782346304808, 5.722714924314928), (2.597782346304808, 5.72271492

In [24]:
cursor.execute('SELECT angleIn1, angleIn5 FROM Parameter WHERE baseValue=(?)', (main, ))
rows = cursor.fetchall()
print(rows)

rows_np = np.array(rows)

phase_all = np.stack([ftx0, ftx1], axis=1)  # shape: (N, 2)

# rows 已转为 np.array，shape: (M, 2)

# 广播：对每个 phase 与所有 row 计算 L1 距离，结果 shape: (N, M)
diff_matrix = np.abs(phase_all[:, None, :] - rows_np[None, :, :])  # shape: (N, M, 2)
distances = np.sum(diff_matrix, axis=2)  # shape: (N, M)

# 每行选最小值的索引
INDEX = np.argmin(distances, axis=1)  # shape: (N,)

[(2.2326421521943827, 5.2624718802425985), (2.3055513798457477, 5.355615082812643), (2.33685621656347, 5.395359756192938), (2.33685621656347, 5.395359756192938), (2.33685621656347, 5.395359756192938), (2.3768338063207666, 5.446965515108369), (2.4289329063547944, 5.51293963783582), (2.4289329063547944, 5.51293963783582), (2.4289329063547944, 5.51293963783582), (2.4515134729821746, 5.54079848645615), (2.4515134729821746, 5.54079848645615), (2.4515134729821746, 5.54079848645615), (2.4515134729821746, 5.54079848645615), (2.4515134729821746, 5.54079848645615), (2.4691625499726775, 5.56351418653956), (2.5314805478666136, 5.641494852180908), (2.5314805478666136, 5.641494852180908), (2.5314805478666136, 5.641494852180908), (2.5760474010453254, 5.696140186967039), (2.5760474010453254, 5.696140186967039), (2.5760474010453254, 5.696140186967039), (2.5760474010453254, 5.696140186967039), (2.5760474010453254, 5.696140186967039), (2.597782346304808, 5.722714924314928), (2.597782346304808, 5.72271492

In [1]:
import numpy as np
import sqlite3
import os
from scipy.spatial import cKDTree

main = 0

base_dir = os.getcwd()
DB_PATH = os.path.join(base_dir, "data", "Main.db")

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

cursor.execute('SELECT * FROM BaseParameter WHERE baseValue=(?)', (main,))
row = cursor.fetchone()

single = row[1]
h = row[2]

N = int(250) * 2
r = N * single
x = np.linspace(-(N / 2), (N / 2), N)

lambda0 = 0.8e-6
lambda1 = 0.6e-6
f = 60e-6

# 计算相位并规整到[0, 2π)
ftx0 = -(2 * np.pi) / lambda0 * (np.sqrt(x**2 + f**2) - f)
ftx0 = np.mod(ftx0, 2 * np.pi)  # wrapTo2Pi

ftx1 = -(2 * np.pi) / lambda1 * (np.sqrt(x**2 + f**2) - f)
ftx1 = np.mod(ftx1, 2 * np.pi)  # wrapTo2Pi

cursor.execute('SELECT angleIn1, angleIn5 FROM Parameter WHERE baseValue=(?)', (main,))
rows = cursor.fetchall()

# 向量化处理开始
# 将数据库行转换为NumPy数组
points = np.array(rows)  # 形状为 (M, 2)，M是数据库行数

# 构建查询点矩阵 (N, 2)
query_points = np.column_stack((ftx0, ftx1))

# 使用cKDTree进行曼哈顿距离(L1)最近邻搜索
tree = cKDTree(points, leafsize=40)  # 调整leafsize优化性能
distances, indices = tree.query(query_points, p=1)  # p=1 表示曼哈顿距离
INDEX = indices

# 关闭数据库连接
conn.close()

print(INDEX)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0  14
   0   0  39   0   0 100   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   2   0   0  30   0   0  46   0   0 407   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   2   0   0  30   0   0  46   0
   0 407   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   2
   0   0  30   0   0  46   0   0 407   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   1   0   0  30   0   0  39   0   0 100   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   2   0   0  16   0
   0  46   0   0 407   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   2   0   0  16   0   0  46   0   0 407   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   2   0   0  16   0   0 146   0   0
 407   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   2   0
   0  16   0   0  46   0   0 100   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   2   0   0  18   0   0 146   

In [1]:
import matlab.engine

eng = matlab.engine.start_matlab()